In [17]:
import pyoti.core as coti
import pyoti.sparse as oti
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import sympy as sym

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Computer Modern Roman",
    "font.size": 12,
    "font.sans-serif": ["Helvetica"]
})

np.set_printoptions(linewidth=120)

In [2]:
def fun2(X,r,m,b):
    #INTERSECTION BETWEEN CURVES.
    f = oti.zeros((2,1))
    x = X[0,0].copy()
    y = X[1,0].copy()
    f[0,0] = x**2 + y**2 - r**2 
    f[1,0] = y - ( m * x + b )
    return f

def Jfun2(X,func,verbose,r,m,b):
    #INTERSECTION BETWEEN CURVES.
    J = oti.zeros((2,2))
    x = X[0,0].copy()
    y = X[1,0].copy()
    J[0,0] = 2 * x 
    J[1,0] = - m 
    J[0,1] = 2 * y 
    J[1,1] = 1
    return J

def fun2_curves(x,r,m,b):
    #CURVES.
    y = oti.zeros((x.shape[0],3))
    y[:,0] =  oti.sqrt(r**2 - x**2)
    y[:,1] = -oti.sqrt(r**2 - x**2)
    y[:,2] = m * x + b
    return y


def intersect_pts(r,m,b,alg=oti):
    #CURVES.
        
    #     x[0,0] = (-m*b + oti.sqrt( (m*b)**2 - (m+1)*(b**2-r**2) ) )/( (m+1) )
    #     x[1,0] = (-m*b - oti.sqrt( (m*b)**2 - (m+1)*(b**2-r**2) ) )/( (m+1) )
    x1 = alg.zeros((2,1))
    x1[0,0] = (-b*m + alg.sqrt(-b**2 + m**2*r**2 + r**2))/(m**2 + 1)
    x1[1,0] = m * x1[0,0] + b
    
    x2 = alg.zeros((2,1))
    x2[0,0] = (-b*m - alg.sqrt(-b**2 + m**2*r**2 + r**2))/(m**2 + 1)
    x2[1,0] = m * x2[0,0] + b
        
    return x1,x2

In [3]:
# utils
def newton_otisis(func, jaco, x, tol, maxiter=50, args=(), verbose=False,eps = 1e-16):
    error = 1e30
    n = 0
    nx = x.shape[0]
    dx = oti.zeros( (nx,1), order=x.order)
    
    argv = (func,verbose)+args
    while error > tol:

        if n==maxiter:
            print("Did not converge. ", maxiter, "iters.")
            break
        if verbose:
            print("Iteration {0}".format(n))
        mJ = -jaco(x,*argv)
#         print(mJ.shape)
        f_eval = func(x,*args)
#         print(f_eval.shape)

        
        dx = oti.dot(oti.inv(mJ),f_eval)
        x += dx
        error = oti.norm(dx)/(oti.norm(x)+eps)
        error = (error).get_deriv(0)
        
        if verbose:
            print("\nf")
            print(f_eval)
            print("\nJ")
            print(-mJ)
            print('\ndx')
            print(dx)
            print("\nx i+1")
            print(x)
            print("Error")
            print("{0:.10f}".format(error))
            print("\n"+100*"=")
        n += 1
    print("niter: ",n)
#     print("rel_err:",(oti.norm(dx)/oti.norm(x)).get_deriv(0))
    return x

def j_DF (x, func, *args):
    h = 1e-8
    nx = x.shape[0]
    J = oti.zeros((nx,nx), order=x.order)
    for i in range(nx):
        xh = x.copy()
        xh[i,0]+= h
        dfdxi = (func(xh,*args) - func(x,*args))/h
        for j in range(nx):
            J[j,i] = dfdxi[j,0]
    return J

def j_oti (x, func,verbose, *args):
    nx = x.shape[0]
    active_bases = func(x,*args).get_active_bases()
    if len(active_bases)==0:
        next_base = 1 
    else:
        next_base = max(active_bases) + 1
    # end if 
    
    e_x = oti.zeros((nx,1), order=(x.order+1))
    for i in range(nx):
        e_x[i] = oti.e(next_base + i, order = (x.order+1))  
    # end for 
    J = oti.zeros((nx,nx), order=x.order)
    fun = func(x+e_x, *args) 
    if verbose:
        print('\nf for jacobian')
        print(fun)
    for n in range(nx):
        J_col = fun.extract_deriv(next_base + n)
        
        for j in range(nx):
            J_col = J_col.truncate(next_base + j)
        # end for 
        for m in range(nx):
            J[m,n] = J_col[m,0]
        # end for 
    # end for 
    return J

In [4]:
intersect_pts(4,3,-4,alg=oti)

(matso< shape: (2, 1), 
  - Column 0
 (0,0) 2.4
 (1,0) 3.2
 >,
 matso< shape: (2, 1), 
  - Column 0
 (0,0) 0
 (1,0) -4
 >)

In [5]:
r = 4
m = 3
b = -4

(-m*b+np.sqrt(r*r*(m*m+1)-b*b))/(m*m+1)

2.4

In [6]:
r*r*(m*m+1)-b*b

144

In [7]:
(b+m*np.sqrt(r*r*(m*m+1)-b*b))/(m*m+1)

3.2

In [8]:
# Define known solution:
x1 = 2.4 + oti.e(1) # 2.4,  0
x2 = 3.2 + oti.e(2) # 3.2, -4

X = oti.array([x1,x2])
res = fun2(X,r,m,b)
print(res)

# Define tangent matrix:
T = np.zeros((2,2)) 
T[:,0:1] = res.get_im([1])
T[:,1:2] = res.get_im([2])
T

matso< shape: (2, 1), 
 - Column 0
(0,0)  + 4.8 * e([1]) + 6.4 * e([2])
(1,0) 8.88178e-16 - 3 * e([1]) + 1 * e([2])
>


array([[ 4.8,  6.4],
       [-3. ,  1. ]])

In [13]:
# Perturb input variables to compute the derivatives wanted.
# Using first truncation order 1.
order = 1
r = r.real + oti.e(1,order=order)
m = m.real + oti.e(2,order=order)
b = b.real + oti.e(3,order=order)

# Define the solution values from the real values known.
# No imaginary perturbations!
x1 = x1.real 
x2 = x2.real 

X = oti.array([x1,x2])
X # X has no OTI perturbations yet.

matso< shape: (2, 1), 
 - Column 0
(0,0) 2.4
(1,0) 3.2
>

In [14]:
# Define maximum order of derivative to compute:
order = 3
for p in range(1,order+1):
    
    # Set truncation order for desing and state variables to ordi.
    
    r = r + 0*oti.e(1,order=p)
    m = m + 0*oti.e(1,order=p)
    b = b + 0*oti.e(1,order=p)

    X = X + 0*oti.e(1,order=p)
    
    # Evaluate residual equation:
    # Evaluate the residual at the known solution.
    res = fun2(X,r,m,b)
    


    # Compute p'th order RHS
    rhsp = oti.get_order_im_array(p,res)
    
    # Solve p'th order imdir of solution .
    Xp = np.dot(Tinv,-rhsp)
    
    # Apply imdirs along OTI solution vector X
    oti.set_order_im_from_array(p,Xp,X)
print(X)

matso< shape: (2, 1), 
 - Column 0
(0,0) 2.4 + 0.333333 * e([1]) - 0.64 * e([2]) - 0.266667 * e([3]) + ... 
(1,0) 3.2 + 1 * e([1]) + 0.48 * e([2]) + 0.2 * e([3]) + ... 
>


In [18]:
# Check residual calculation:
res = fun2(X,r,m,b)
for p in range(1,order+1):
    RESp = oti.get_order_im_array(p,res)
    print('Total derivative of order {0}'.format(p))
    print(RESp)

Total derivative of order 1
[[ 0.00000000e+00 -4.44089210e-16  0.00000000e+00]
 [ 2.22044605e-16  4.44089210e-16  1.66533454e-16]]
Total derivative of order 2
[[-1.11022302e-16 -1.11022302e-16  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-3.46944695e-18  4.53341068e-17 -1.38777878e-16 -1.04083409e-17 -2.77555756e-17 -1.73472348e-18]]
Total derivative of order 3
[[ 8.67361738e-19  6.93889390e-18 -5.55111512e-17 -5.55111512e-17 -6.93889390e-18  0.00000000e+00  2.77555756e-17
  -3.46944695e-18  3.46944695e-18  0.00000000e+00]
 [ 8.67361738e-19  1.73472348e-18 -2.34380416e-17  4.16333634e-17  1.73472348e-18  6.93889390e-18  1.38777878e-17
   8.67361738e-19  1.73472348e-18  1.08420217e-19]]
